# Subsample Polygons

### Here we will cut up the drawn polygons into equal sized hexagons

In [ ]:
import geopandas as gpd

In [ ]:
plots_path = '../data/polygons/RegressionRidge_smol.geojson'

plots = gpd.read_file(plots_path)

In [ ]:
plots.crs

In [ ]:
plots = plots.to_crs(26910)   # UTM for meter-based subdivision

In [ ]:
import numpy as np
from shapely.geometry import Polygon, MultiPolygon

def make_hex(center_x, center_y, r):
    '''Flat-top hexagon with circumradius r.'''
    angles = np.deg2rad([0, 60, 120, 180, 240, 300])
    return Polygon([
        (center_x + r * np.cos(a), center_y + r * np.sin(a))
        for a in angles
    ])

def clean_geom(g, min_area):
    '''Fix invalids, remove slivers, collapse multipolygons.'''
    if g.is_empty:
        return None

    # Fix invalid geometries (self-crossing, slivers, etc.)
    g = g.buffer(0)

    if g.is_empty:
        return None

    # If multipolygon, take largest solid piece
    if isinstance(g, MultiPolygon):
        g = max(g.geoms, key=lambda x: x.area)

    # Final sliver removal
    if g.area < min_area:
        return None

    return g


def subdivide_hexagonal(poly, min_area_m2):
    '''
    Subdivide a polygon into equal-area hexagons (flat top) with no gaps.
    Includes sliver removal & geometry cleanup.
    '''
    # Area of flat-top hex: A = (3√3 / 2) * r²
    r = np.sqrt((2 * min_area_m2) / (3 * np.sqrt(3)))

    # Hex spacing (flat-top geometry)
    horiz = 1.5 * r             # center-to-center horizontally
    vert  = np.sqrt(3) * r      # center-to-center vertically

    minx, miny, maxx, maxy = poly.bounds

    hexes = []
    y = miny
    row = 0

    while y <= maxy + vert:
        # offset every other row
        x_start = minx + (0.75 * r if row % 2 else 0)
        
        x = x_start
        while x <= maxx + horiz:
            h = make_hex(x, y, r)
            clipped = poly.intersection(h)

            # Clean + sliver removal injected HERE
            geom = clean_geom(clipped, min_area_m2 * 0.25)

            if geom is not None:
                hexes.append(geom)

            x += horiz
        
        y += vert
        row += 1

    return hexes


In [ ]:
MIN_AREA = 1000  # m²

subplots = []

for idx, row in plots.iterrows():
    poly = row.geometry
    
    
    parent = row.get('Name', f'plot_{idx}')

    parts = subdivide_hexagonal(poly, MIN_AREA)

    n = 0
    for geom in parts:
        
        
        subplots.append({
            'plot_id': parent,
            'geometry': geom
        })
        n += 1

subplots_gdf = gpd.GeoDataFrame(subplots, crs=plots.crs)


In [ ]:
sub_gdf = gpd.GeoDataFrame(subplots_gdf, crs=plots.crs)


In [ ]:
sub_gdf['plot_id'] = sub_gdf.index

In [ ]:
sub_gdf = sub_gdf.to_crs(4326)

In [ ]:
sub_gdf

In [ ]:
sub_gdf.to_pickle('../data/polygons/RegressionRidge_smol_smol.pkl')